In [12]:
import os
from os import path,makedirs,listdir
import pandas as pd
import numpy as np
import difflib as diff
import pydicom
from pydicom import dcmread




data_path = "/mnt/data/romy/hypnomed/MRI_raw"
heudiconv_path = os.path.join(data_path,'BIDS/.heudiconv')
dicom_path = os.path.join(data_path,'DICOM')
info_output = os.path.join(data_path,'fmri_infos')
info_output_git = '/home/romy.beaute/projects/hypnomed/code/Visualize/infos'

dtypes = ['rs','3DT1','3DT2','t2star']

save_csv = True


In [13]:
def sub_indexes_df(dicom_path):
    id_dict =  {}
    heudi_list = sorted(os.listdir(heudiconv_path))
    dicom_list = sorted(os.listdir(dicom_path))
    dicom_list_idx = [idx.split('-')[1] for idx in dicom_list] #get the id number of each subject

    for idc,idh in zip(dicom_list,heudi_list):
        id_dict[idc] = {}
        #df = df_tsv(subject_idx=idh,dtype='t2star')  
        #id_dict[idc]['neucose_id']= df.patient_id[1]
        #id_dict[idc]['age']= df.patient_age
        dcm_files = [file for file in os.listdir('/mnt/data/romy/hypnomed/MRI_raw/DICOM/{}/ses-001/1_AUTOALIGN/'.format(idc)) if not file.startswith('._')]
        dcm_file = os.path.join((os.path.join(dicom_path,'{}/ses-001/1_AUTOALIGN'.format(idc))),dcm_files[0]) #take the first dcm file corresp to the 1_AUTOALIGN folder of sub idc
        ds = pydicom.dcmread(dcm_file,force=True)
        try:
            id_dict[idc]['sub_id']= ds.PatientName
            id_dict[idc]['neucose_id'] = ds.PatientID
            id_dict[idc]['image_type']= ds.ImageType
            id_dict[idc]['age']= ds.PatientAge #for sanitary check
        except:
            pass

   
    df_subid = pd.DataFrame.from_dict(id_dict,orient='index')

    map_id_neuco = dict()

    for _,(id,neuco) in enumerate(zip(list(df_subid.sub_id),list(df_subid.neucose_id))):
        map_id_neuco[neuco] = id

    return df_subid,map_id_neuco



In [14]:
def df_tsv(subject_idx,dtype):

    path_tsv = os.path.join(heudiconv_path,'{}/ses-001/info/dicominfo_ses-001.tsv'.format(subject_idx))
    df_tsv = pd.read_csv(path_tsv,sep='\t')

    filename = [type for type in df_tsv.dcm_dir_name if dtype in type]
    df = df_tsv.loc[df_tsv["dcm_dir_name"].isin(filename)]

    return df.set_axis([idx.split('_')[0] for idx in df['dcm_dir_name']],axis='index')  #set the index with the identifiant of the subject



def heudiconv_csv(heudiconv_path,save_csv,map_id_neuco):

    subject_idxs = [sub for sub in (sorted(os.listdir(heudiconv_path)))] #can choose from there the index of the subject we want to get infos 
    dtypes = ['rs','3DT1','3DT2','t2star']

    subjects_dict = {}
    for _,sub in enumerate(subject_idxs):
        subjects_dict['sub-{}'.format(sub)]={}
        for _,dtype in enumerate(dtypes):
            subjects_dict['sub-{}'.format(sub)][dtype]=df_tsv(sub,dtype)
            df = df_tsv(sub,dtype)
            subjects_dict['sub-{}'.format(sub)]['neucose_id'] = df.patient_id[1]
            subjects_dict['sub-{}'.format(sub)]['sub_id'] = [value  for (key, value) in map_id_neuco.items() if key == df.patient_id[1]][0]
            

            if save_csv:
                df.to_csv(info_output+'/sub-{}/{}_{}_diffinfos.csv'.format(sub,sub,dtype)) #write csv 
            
    df_heudiconv = pd.DataFrame.from_dict(subjects_dict, orient='index')
    return df_heudiconv

In [15]:
import csv

df_subid,map_id_neuco = sub_indexes_df(dicom_path)

if save_csv:
    df_subid.to_csv(info_output_git+'/sub_ids.csv') #write csv 

    with open(info_output_git+'/map_subid_neuco.csv', 'w') as f:
        for key in map_id_neuco.keys():
            f.write("%s, %s\n" % (key, map_id_neuco[key]))

In [16]:
df_heudiconv = heudiconv_csv(heudiconv_path,save_csv,map_id_neuco)
df_heudiconv.to_csv(info_output_git+'/heudiconv_infos.csv')
df_heudiconv.head()

,rs,neucose_id,sub_id,3DT1,3DT2,t2star
sub-01,total_files_till_now ...,NEUCOSE_ELSES07842,"(E, H, 0, 7, 0)",total_files_till_now ...,total_files_till_now ...,total_files_till_now ...
sub-02,total_files_till_now ...,NEUCOSE_LECPA07862,"(L, P, 0, 8, 7)",total_files_till_now ...,total_files_till_now ...,total_files_till_now ...
sub-03,total_files_till_now ...,NEUCOSE_HARMA07939,"(H, M, 0, 7, 6)",total_files_till_now ...,total_files_till_now ...,total_files_till_now ...
sub-04,total_files_till_now ...,NEUCOSE_BUEAL07925,"(B, A, 0, 4, 3)",total_files_till_now ...,total_files_till_now ...,total_files_till_now ...
sub-05,total_files_till_now ...,NEUCOSE_BLUAL07960,"(B, A, 0, 9, 3)",total_files_till_now ...,total_files_till_now ...,total_files_till_now ...


### Infos for a single subject & single type

In [17]:
subject_idx = '16' #can be any of one from the sub_idxs list
dtype = 'rs'
show_diff = False

#types of infos we can have access to 
keys = [key for key in df_heudiconv.loc['sub-{}'.format(subject_idx),'{}'.format(dtype)].keys()]
print(keys)



['total_files_till_now', 'example_dcm_file', 'series_id', 'dcm_dir_name', 'unspecified2', 'unspecified3', 'dim1', 'dim2', 'dim3', 'dim4', 'TR', 'TE', 'protocol_name', 'is_motion_corrected', 'is_derived', 'patient_id', 'study_description', 'referring_physician_name', 'series_description', 'sequence_name', 'image_type', 'accession_number', 'patient_age', 'patient_sex', 'date', 'series_uid']


In [18]:
#1st way : 
df = df_tsv(subject_idx,dtype)
series_uid = list()
series_uid.extend(s for s in df.series_uid)

In [158]:
#2nd way : 
df = df_heudiconv.loc['sub-{}'.format(subject_idx),dtype if dtype is not None else ':']
df.patient_id

# if save_csv:
#     df.to_csv(info_output+'/{}_{}_diffinfos.csv'.format(subject_idx,dtype)) #write csv 


14    NEUCOSE_ELSES07842
15    NEUCOSE_ELSES07842
Name: patient_id, dtype: object

### Spot the differences for one specific subject and one specific type 

In [159]:
if show_diff:
    for key in df.keys():
        print('\n{}'.format(key))
        for id in df.index:
            print('- file {} : {}'.format(id,df.loc[id]['{}'.format(key)]))

### Spot the differences for all subjects and one specific type 

In [11]:
# Example with only one subject : 2 ways 
show_diff = True
dtype = 'rs'
sub_idxs = [id.split('-')[1] for id in list(df_heudiconv.index)]

dict_totfiles = {}
sec_rs = list () #store the values for rs2

#2nd way : 

if show_diff:
    f=open(info_output_git+'/{}_infos_all.txt'.format(dtype),'w')

    f.write('***** {} informations *****\n'.format(dtype))

    for subject_idx in sub_idxs:
        dict_totfiles[subject_idx]={}
        list_files = list()
        

        df = df_heudiconv.loc['sub-{}'.format(subject_idx),dtype]
        for key in df.keys():
            if key == 'total_files_till_now':
                f.write('\n sub-{} : {}\n {}:\n'.format(subject_idx,df_heudiconv.loc['sub-{}'.format(subject_idx)].sub_id,key))
                #print('\n sub-{} : {}\n {}:\n'.format(subject_idx,df_heudiconv.loc['sub-{}'.format(subject_idx)].sub_id,key))
                for _,id in enumerate(df.index):
                    list_files.append(df.loc[id]['{}'.format(key)])
                    f.write('- file {} : {}\n'.format(id,df.loc[id]['{}'.format(key)]))
                    if _ == 1:
                        sec_rs.append(df.loc[id]['{}'.format(key)])
                    #print('- file {} : {}'.format(id,df.loc[id]['{}'.format(key)]))
                dict_totfiles[subject_idx]=list_files
            
    f.close()


    
#get the lower bound 
minrs1 = min([min(l) for l in dict_totfiles.values()])
print('minrs1 :',minrs1)
maxrs1 = max([min(l) for l in dict_totfiles.values()])
print('maxrs1 :',maxrs1)
print(10*'-')

#get the middle bound 
minrs2 = min([l for l in sec_rs])
print('minrs2 :',minrs2)
maxrs2 = max([l for l in sec_rs])
print('maxrs2 :',maxrs2)
print(10*'-')

#get the lower bound 
minrs3 = min([max(l) for l in dict_totfiles.values()])
print('minrs3 :',minrs3)
maxrs3 = max([max(l) for l in dict_totfiles.values()])
print('maxrs3 :',maxrs3)



minrs1 : 384
maxrs1 : 601
----------
minrs2 : 1200
maxrs2 : 2724
----------
minrs3 : 2385
maxrs3 : 3109


In [204]:
df.dcm_dir_name

8      8_RUNx_rs_3mmiso_TR2.4_44cpe
16    16_RUNx_rs_3mmiso_TR2.4_44cpe
22    22_RUNx_rs_3mmiso_TR2.4_44cpe
Name: dcm_dir_name, dtype: object

In [198]:
#get the middle bound 
for s in sec_rs:
    if s >= 2300:
        print(s)


2724


# Multiple files

In [ ]:
def get_dicom_infos(heudiconv_path,map_id_neuco):


    heudi_list = sorted(os.listdir(heudiconv_path)) #get the indexes of the subjects
    

    bids_info_paths = list()
    for _,sub in enumerate(sorted(os.listdir(heudiconv_path))):
        bids_info_paths.append(os.path.join(heudiconv_path,'{}/ses-001/info/dicominfo_ses-001.tsv'.format(sub))) #permet de récupérer les paths des infos pour chaque sujet
    

    #Create 3 dict : 
    dcm_dict_indxs = dict() #store the tsv in df format for each subject
    infos_dict_indxs = dict() #strores the infos of each dicom
    dcm_dict_paths = dict() #stores the paths of the files of interest

    for _,(sub_indx,bid_tsv) in enumerate(zip(heudi_list,bids_info_paths)):
        # if print_paths: #print the paths for sanitary check
        #     print(70*'-')
        #     print('Getting infos for subject n°{}, stored in {}'.format(sub_indx,bid_tsv))
        #     print(70*'-')

        #1st dictionnary
        infos = pd.read_csv(bid_tsv,sep='\t')
        infos_dict_indxs["Sub-{}".format(sub_indx)]=infos 
        print(infos.patient_id)

        #2nd dictionnary
        dcm_dict_indxs["Sub-{}".format(sub_indx)]={}
        dcm_dict_indxs["Sub-{}".format(sub_indx)]["sub_code"] = infos.patient_id


        #3rd dictionnary
        dcm_dict_paths["Sub-{}".format(sub_indx)]={} 


        
        # dcm_dict_indxs["Sub-{}"]["sub_code"] = {}

        t2star_dcm = [type for type in infos.dcm_dir_name if 't2star' in type]
        rs_dcm = [type for type in infos.dcm_dir_name if 'rs' in type]
        t1_dcm = [type for type in infos.dcm_dir_name if '3DT1' in type]
        t2_dcm = [type for type in infos.dcm_dir_name if '3DT2' in type]

        dcm_dict = {'RS':rs_dcm,
                    'T1':t1_dcm,
                    'T2':t2_dcm,
                    'T2*':t2star_dcm}

        
        rs_indxs = list()
        t1_indxs = list()
        t2_indxs = list()
        t2star_indxs = list()
        
        for _,(key,val) in enumerate(zip(dcm_dict.keys(),dcm_dict.values())):

            # if print_paths: #print the names of the files for sanitary check
            #     print("{} files : {}\n".format(key,val))

            if key == 'RS': #get the indx number that corresponds to each RS file :
                rs_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['RS'] = rs_indxs
                dcm_dict_paths["Sub-{}".format(sub_indx)]['RS'] = val

            elif key == 'T1':
                t1_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['T1'] = t1_indxs
                dcm_dict_paths["Sub-{}".format(sub_indx)]['T1'] = val

            elif key == 'T2':
                t2_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['T2'] = t2_indxs
                dcm_dict_paths["Sub-{}".format(sub_indx)]['T2'] = val

            elif key == 'T2*':
                t2star_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['T2*'] = t2star_indxs
                dcm_dict_paths["Sub-{}".format(sub_indx)]['T2*'] = val
        
    dcm_df = pd.DataFrame.from_dict(dcm_dict_indxs, orient='index') #convert the dictionnary into dataframes
    return dcm_df,infos_dict_indxs,dcm_dict_paths


dcm_df,infos_dict_indxs,dcm_dict_paths = get_dicom_infos(heudiconv_path,map_id_neuco)

In [ ]:
#sanitary check for heuristic.py  
total_files_till_now = 2000
if total_files_till_now <100:
    print(1)
elif total_files_till_now >2000:
    print(1)
elif ((total_files_till_now >= 1200) and (total_files_till_now <= 2300)):
    print(1)

1


In [ ]:
mri_database = pd.read_csv(info_output_git+'/MRI_database_logs.csv',delimiter=';')
mri_database.set_index('sub code')

,sub number,version,ER runs,RS runs,comments,reorient,movement,quality check preproc (does the wT1 match the wwraRUN and MNI),Exclude >1.5mm movement,ER Level 1,RS level 1
sub code,,,,,,,,,,,
KA001,1.0,B,12 20 26,8 18 24,NaN,x,ok,x,NaN,ok,ok
AV005,5.0,A,10 18 24,8 16 22,excl run M,x,several big spikes in run M,x,Run M,run M not visible,too much movement
AV007,7.0,A,10 18 24,8 16 22,NaN,x,ok,x,NaN,ok,ok
JC014,14.0,B,10 16 24,8 14 22,NaN,x,ok,x,NaN,ok,ok
GM021,21.0,B,10 16 24,8 14 22,t2 refaite à la fin du protocole,x,ok,x,NaN,ok,much movement in M
PM024,24.0,A,10 18 24,8 16 22,resp signal fluctuant,x,ok,x,NaN,ok,very clear acti
CS025,25.0,B,10 18 24,8 16 22,NaN,x,ok,x,NaN,ok,ok
NJ029,29.0,B,10 18 24,8 16 22,NaN,x,ok,x,NaN,ok,ok (activation in eyes)
HR034,34.0,B,10 16 24,8 14 22,petit mouvement dans dernière ER bloc 3 (micro...,x,ok,x,NaN,ok,movement in M


In [88]:
ds_subpb.patient_id

30    NEUCOSE_CAMFA08095
31    NEUCOSE_CAMFA08095
Name: patient_id, dtype: object

In [114]:
check_list = ['LA060','CF081','GM021','EH070'] #subjects with problems, check in heudiconv

for s in df_heudiconv.index:
    if df_heudiconv.loc[s].sub_id in check_list:
        subject_idx = s.split('-')[1]
        print(60*'-')
        print(df_heudiconv.loc[s].sub_id)
        for dtype in dtypes:
            ds_subpb = df_tsv(subject_idx,dtype)
            neucose_id = ds_subpb.patient_id
            print(10*'- - - ')
            print('\n ***** {} *****'.format(dtype))
            print('{} : {} \n'.format(s,neucose_id[0]))
            #print(ds_subpb.patient_id)
            print(ds_subpb.dcm_dir_name)


------------------------------------------------------------
EH070
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

 ***** rs *****
sub-01 : NEUCOSE_ELSES07842 

8      8_RUNx_rs_3mmiso_TR2.4_44cpe
16    16_RUNx_rs_3mmiso_TR2.4_44cpe
22    22_RUNx_rs_3mmiso_TR2.4_44cpe
Name: dcm_dir_name, dtype: object
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

 ***** 3DT1 *****
sub-01 : NEUCOSE_ELSES07842 

20    20_anat_3DT1_0.9iso
21    21_anat_3DT1_0.9iso
26    26_anat_3DT1_0.9iso
27    27_anat_3DT1_0.9iso
Name: dcm_dir_name, dtype: object
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

 ***** 3DT2 *****
sub-01 : NEUCOSE_ELSES07842 

14    14_anat_3DT2_0.9iso
15    15_anat_3DT2_0.9iso
Name: dcm_dir_name, dtype: object
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

 ***** t2star *****
sub-01 : NEUCOSE_ELSES07842 

12    12_anat_t2star_2iso
13    13_anat_t2star_2iso
Name: dcm_dir_name, dtype: object
-----------------------------------------

In [82]:
check_list_subcode = 'LA060' #subjects with problems, check in heudiconv
dtype = 't2star'

for s in df_heudiconv.index:
    if df_heudiconv.loc[s].sub_id == check_list_subcode:
        subject_idx = s.split('-')[1]
        ds_subpb = df_tsv(subject_idx,dtype)
        print(ds_subpb.dcm_dir_name)


12    12_anat_t2star_2iso
13    13_anat_t2star_2iso
Name: dcm_dir_name, dtype: object
